In [ ]:
import os
from dotenv import load_dotenv
import openai
from dataclasses import dataclass, field
from typing import List
import json
from system_prompts import SYSTEM_PROMPT_FIRST_SEARCH, SYSTEM_PROMPT_FIRST_SUMMARY, SYSTEM_PROMPT_REFLECTION

### 加载环境变量和预定义函数及数据结构

In [3]:
load_dotenv()

@dataclass
class Search:
    url: str = ""
    content: str = ""
    
@dataclass
class Research:
    search_history: List[Search] = field(default_factory=list)
    latest_summary: str = ""
    reflection_iteration: int = 0

@dataclass
class Paragraph:
    title: str = ""
    content: str = ""
    research: Research = field(default_factory=Research)

@dataclass
class State:
    report_title: str = ""
    paragraphs: List[Paragraph] = field(default_factory=list)


def clean_json_tags(json_str):
    return json_str.replace("```json", "").replace("```", "").strip()

def update_state_with_search_results(search_results, idx_paragraph, state):

    for search_result in search_results["results"]:
        search = Search(url=search_result["url"], content=search_result["content"])
        state.paragraphs[idx_paragraph].research.search_history.append(search)
    return state



### 初始化状态

In [4]:
STATE = State()

### 报告大纲智能体

In [5]:
class ReportStructureAgent:
    def __init__(self):
        self.llm = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

    def get_system_prompt(self):        
        output_schema_report_structure = {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "title": {"type": "string"},
                        "content": {"type": "string"}
                    }
                }
            }

        example_output_report_structure = """
```json
[
{
    "title": "智能体的定义",
    "content": "智能体是具有感知、决策、执行和学习能力的自主系统。"
},
{
    "title": "智能体的分类",
    "content": "智能体可以分为以下几类："
}
]
```
"""

        system_prompt = f"""
你是一个深度调研助手。针对一个查询任务，请规划一份报告的结构以及应包含的段落内容。
请确保段落的排列顺序合理。
结构制定完成后，你将获得工具来分别为每个部分进行网页搜索和反思。
请按照以下 JSON 格式定义的格式输出结果：

<OUTPUT JSON SCHEMA>
{json.dumps(output_schema_report_structure, indent=2)}
</OUTPUT JSON SCHEMA>

标题（title）和内容（content）属性将用于后续的深入研究。
请确保输出是一个符合上述 JSON 格式定义的 JSON 对象。
只返回 JSON 对象，不要附加任何解释或额外文本。

<EXAMPLE OUTPUT>
{example_output_report_structure}
</EXAMPLE OUTPUT>
"""

        return system_prompt
        
    def run(self, input):
        response = self.llm.chat.completions.create(
            model="gpt-4o-2024-08-06",
            messages=[
                {"role": "system", "content": self.get_system_prompt()},
                {"role": "user", "content": input}
            ],
            stream=False
        )

        report_structure = json.loads(clean_json_tags(response.choices[0].message.content))

        return report_structure

In [6]:
research_structure_agent = ReportStructureAgent()

research_input = "请帮我调研一下web3.0的开发"

report_structure = research_structure_agent.run(research_input)
for paragraph in report_structure:
    STATE.paragraphs.append(Paragraph(title=paragraph["title"], content=paragraph["content"]))

print("报告大纲：")
for i, paragraph in enumerate(STATE.paragraphs):
    print(f"第{i}个段落标题: {paragraph.title}") 
    print(f"第{i}个段落内容: {paragraph.content}")

报告大纲：
第0个段落标题: Web3.0的定义和背景
第0个段落内容: 介绍Web3.0的概念及其起源，区分其与Web1.0和Web2.0的差异，强调去中心化和用户自主权的重要性。
第1个段落标题: Web3.0的关键技术
第1个段落内容: 探讨区块链、智能合约、去中心化应用（DApps）、分布式共识机制等Web3.0的核心技术及其运作方式。
第2个段落标题: Web3.0的开发工具和框架
第2个段落内容: 列举支持Web3.0开发的主要工具和框架，如Truffle、React、Web3.js等，并说明其功能和使用场景。
第3个段落标题: 开发Web3.0应用的流程
第3个段落内容: 介绍如何从概念到实现进行Web3.0应用开发的基本流程，包括需求分析、智能合约编写、前后端集成等步骤。
第4个段落标题: Web3.0的安全性和隐私
第4个段落内容: 分析Web3.0在安全性和隐私保护方面的挑战和对策，探讨常见的安全漏洞及如何防范。
第5个段落标题: Web3.0的行业应用场景
第5个段落内容: 展示Web3.0技术在金融、供应链、社交媒体、数字身份等行业的应用案例，分析其带来的创新和优势。
第6个段落标题: Web3.0开发的挑战与机遇
第6个段落内容: 讨论目前Web3.0在开发过程中面临的技术、法律、用户体验等挑战，以及其带来的市场机遇和未来趋势。
第7个段落标题: 如何学习和加入Web3.0社区
第7个段落内容: 提供学习Web3.0开发的资源和机会，介绍相关的社区、论坛和培训课程，帮助开发者快速上手和深度参与。


### 搜索智能体

In [25]:
import tavily

class SearchAgent:
    def __init__(self):
        self.llm = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        self.search_tool = tavily.TavilyClient(api_key=os.getenv("TAVILY_API_KEY")
    )

    def get_system_prompt(self):
        

        input_schema_first_search = {
                    "type": "object",
                    "properties": {
                        "title": {"type": "string"},
                        "content": {"type": "string"}
                    }
                }

        output_schema_first_search = {
                    "type": "object",
                    "properties": {
                        "search_query": {"type": "string"},
                        "reasoning": {"type": "string"}
                    }
                }


        system_prompt = f"""
你是一个深度研究助手。你将会被提供一段报告中的段落、它的标题以及期望内容，格式如下的 JSON 格式定义：

<INPUT JSON SCHEMA>
{json.dumps(input_schema_first_search, indent=2)}
</INPUT JSON SCHEMA>

你可以使用一个网页搜索工具，该工具接受一个 search_query 作为参数。
你的任务是对该主题进行思考，并提供一个最优的网页搜索查询（请使用中文），以丰富你当前的知识。
请按照以下的 JSON 格式定义来格式化你的输出：

<OUTPUT JSON SCHEMA>
{json.dumps(output_schema_first_search, indent=2)}
</OUTPUT JSON SCHEMA>

请确保你的输出是一个符合上述输出 JSON 架构定义的 JSON 对象。
只返回 JSON 对象，不要附加任何解释或其他文本。
"""

        return system_prompt

    def run(self, input):                
        response = self.llm.chat.completions.create(
            model="gpt-4o-2024-08-06",
            messages=[
                {"role":"system","content":SYSTEM_PROMPT_FIRST_SEARCH},
                {"role":"user","content":json.dumps(input)}
                ],
            temperature=1
            )
        
        search_arguments = json.loads(clean_json_tags(response.choices[0].message.content))

        return search_arguments
    
    def search(self, search_query, include_raw_content=True, max_results=5):
        search_results = self.search_tool.search(
                query=search_query,
                include_raw_content=include_raw_content,
                max_results=max_results
                )
        return search_results


In [ ]:
search_agent = SearchAgent()

search_input = {
    "title": STATE.paragraphs[0].title,
    "content": STATE.paragraphs[0].content
}


search_arguments = search_agent.run(search_input)
print(search_arguments["search_query"])

search_results = search_agent.search(search_arguments["search_query"])

STATE = update_state_with_search_results(search_results, 0, STATE)

for search in STATE.paragraphs[0].research.search_history:
    print(search.url)
    print(search.content)
    break


Web3.0 定义 起源 去中心化 用户自主权 Web1.0 Web2.0 区别
https://www.cnblogs.com/listen80/p/18304967
Web1.0：虽然内容由少数人创建，但平台的中心化程度相对较低。 · Web2.0：高度中心化，平台拥有绝对的控制权和规则制定权。 · Web3.0：去中心化的架构，减少了对


[{'url': 'https://www.cnblogs.com/listen80/p/18304967',
  'title': 'Web1.0、Web2.0 和Web3.0 的区别- 彩虹刀法 - 博客园',
  'content': 'Web1.0：虽然内容由少数人创建，但平台的中心化程度相对较低。 · Web2.0：高度中心化，平台拥有绝对的控制权和规则制定权。 · Web3.0：去中心化的架构，减少了对',
  'score': 0.78154784,
  'raw_content': '![](https://img2024.cnblogs.com/blog/35695/202506/35695-20250620221146444-645204917.webp)\n![博客园logo](//assets.cnblogs.com/logo.svg)\n![搜索](//assets.cnblogs.com/icons/search.svg)\n![搜索](//assets.cnblogs.com/icons/enter.svg)\n![搜索](//assets.cnblogs.com/icons/search.svg)\n![搜索](//assets.cnblogs.com/icons/search.svg)\n![写随笔](//assets.cnblogs.com/icons/newpost.svg)\n![我的博客](//assets.cnblogs.com/icons/myblog.svg)\n![短消息](//assets.cnblogs.com/icons/message.svg)\n![简洁模式](//assets.cnblogs.com/icons/lite-mode-on.svg)\n![用户头像](//assets.cnblogs.com/icons/avatar-default.svg)\n![返回主页](/skins/custom/images/logo.gif)\n\n# [看完会变帅的哦](https://www.cnblogs.com/listen80)\n\n## 问题不解决就会越来越大\n\n# [Web1.0、Web2.0 和 Web3.0 的区别](https://www.cnblogs.com/listen80/p/1

### 段落输出智能体

In [ ]:
input_json_search_results = {
    "title": STATE.paragraphs[0].title,
    "content": STATE.paragraphs[0].content,
    "search_query": search_input["search_query"],
    "search_results": [ result["raw_content"] for result in search_output["results"] if result["raw_content"]]
}


response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=[
        {"role":"system","content":SYSTEM_PROMPT_FIRST_SUMMARY},
        {"role":"user","content":json.dumps(input_json_first_search)}
        ],
    temperature=1
    )

paragraph_output = json.loads(response.choices[0].message.content)

print(paragraph_output["paragraph_latest_state"])

### 反思智能体

In [ ]:
input_json_reflection = {
    "title": STATE.paragraphs[0].title,
    "content": STATE.paragraphs[0].content,
    "paragraph_latest_state": paragraph_output["paragraph_latest_state"]
}

response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=[
        {"role":"system","content":SYSTEM_PROMPT_REFLECTION},
        {"role":"user","content":json.dumps(input_json_reflection)}
        ],
    temperature=1
    )
    
reflection_output = json.loads(response.choices[0].message.content)

print(reflection_output["search_query"])
